# 1. Ödev

Bu ödevde herhangi bir şekilde varolan sofistike kütüphaneler kullanılamaz. Sadece rastgele sayı üretmek için kütüphane kullanabilirsiniz. Dokümanları vektörize etmek, uzaklık hesapları yapmak vs. gibi her şey için gereken kodu kendiniz yazmalısınız.

Önemli not: ChatGpt'ye ben de kod yazdırabiliyorum :) ChatGPT'den alınan cevaplar kopya muamelesi görecektir.

Ödeviniz Python 3 ile implement ediniz

**Soru 1** Bu soruda locality sensitive hashing (LSH) metodu yazacaksınız. Metodun çağrımı şu şekildedir: lsh(klasor_adi, k, b, r, similarity_threshold). 
- klasor_adi indeksin kurulacağı dosyaları tutan klasörün adresidir. Klasörün içinde birçok txt dosyası var olduğunu varsayabilirsiniz.
- k 'k-shingle'in k değeridir. 
- b band sayısının değeridir.
- r her banddaki satır (row) sayısıdır. 

Verilen  klasördeki dosyaları okuyarak LSH sistemi kurun. Bunun için her kelime bir token olacaktır. Öncelikle her kelime küçük harflere çevrilmeli ve noktalama işaretleri çıkarılmalı. Eğer k = 2 ise, “Ali okula gitti.” cümlesindeki ‘shingle’lar “ali okula” ve “okula gitti” olmalı. 

MinHash imzaları için 10 tane hash fonksiyonu kullanınız. 

Not: Sorunun cevabını kontrol ederken ben size verilen dosyalar ile aynı formatta olan farklı dosyalar kullanacağım.


In [2]:
def zero_vec(length): # length uzunluğunda 0'lardan oluşan bir vektör döndürür
    vec = []
    for i in range(length):
        vec.append(0)
    return vec

In [3]:
def create_hash_functions(num_hash_functions, num_of_documents):
    import random
    hash_functions = []
    for i in range(num_hash_functions):
        a = random.randint(1, num_of_documents)
        b = random.randint(1, num_of_documents)
        hash_functions.append([a, b])
    return hash_functions

In [3]:
create_hash_functions(10, 100)

[[11, 71],
 [21, 9],
 [40, 8],
 [23, 47],
 [88, 21],
 [17, 11],
 [89, 63],
 [99, 9],
 [13, 64],
 [92, 69]]

In [52]:
def lsh(klasor_adi, k, b, r):
    import os
    import string
    import sys

    matrix_represent = dict() # shingle'larin matrixteki temsilini tutacak

    dosya_isimleri = os.listdir(klasor_adi) # klasorun icindeki tum dosyalarin adini aliyoruz

    # tüm dosyaları tek tek oku
    for dosya in dosya_isimleri: 
        content = open(klasor_adi + "/" + dosya, "r").read()
        content = content.replace("\n", " ")
        content = content.lower()
        content = content.translate(str.maketrans('', '', string.punctuation))
        content = " ".join(content.split())        
        content = content.split(" ")
        for i in range(len(content) - k):
            shingle =  content[i:i + k]
            shingle = " ".join(shingle)
            if shingle not in matrix_represent.keys():
                vec = zero_vec(len(dosya_isimleri))
                vec[dosya_isimleri.index(dosya)] = 1
                matrix_represent[shingle] = vec
            else:
                matrix_represent[shingle][dosya_isimleri.index(dosya)] = 1


    hash_functions = create_hash_functions(10, len(dosya_isimleri)) # 10 tane hash fonksiyonu oluşturuluyor

    indexes = range(1,len(dosya_isimleri)+1) # dosya isimlerinin indexleri

    # hash fonksiyonlarıyla indexleri çarpıp modunu alıyoruz
    hash_indexes = dict()

    for i in range(10):
        hash_indexes[i] = []
        for j in indexes:
            hash_indexes[i].append((hash_functions[i][0] * j + hash_functions[i][1]) % len(dosya_isimleri))

    signature_matrix = dict() # signature matrix oluşturuluyor

    for i in range(10):
        signature_matrix[i] = []
        for j in matrix_represent.keys():
            signature_matrix[i].append(min([hash_indexes[i][k] for k in range(len(hash_indexes[i])) if matrix_represent[j][k] == 1]))

    
    # signature matrix'i b ve r değerlerine göre bölüyoruz
    signature_matrix_bolunmus = dict()
    for i in range(b): # signature matrix'i b'ye bölüyoruz
        signature_matrix_bolunmus[i] = []
    
        for doc in range(len(signature_matrix[0])): # her bir doküman için
            temp = []
            for j in range(r):
                temp.append(signature_matrix[i+j][doc])
            signature_matrix_bolunmus[i].append(temp)

    return signature_matrix_bolunmus

In [55]:
a = lsh("docs2", 2,2,5)

In [56]:
a

{0: [[0, 6, 0, 0, 0],
  [4, 6, 0, 2, 3],
  [4, 6, 0, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [2, 6, 1, 1, 1],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [0, 6, 0, 0, 0],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 0, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [0, 6, 0, 0, 0],
  [0, 6, 0, 0, 0],
  [0, 6, 0, 0, 0],
  [0, 6, 0, 0, 0],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [0, 6, 0, 0, 0],
  [0, 6, 1, 1, 0],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [0, 6, 1, 0, 0],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [4, 6, 1, 2, 3],
  [0, 6, 1, 0, 0],
  [2, 6, 

In [31]:
signature_matrix = []

for i in range(len(a[0])):
    temp = []
    for j in range(len(a)):
        temp.append(a[j][i])
    signature_matrix.append(temp)

In [33]:
#find similar documents
similar_docs = []
for i in range(len(signature_matrix)):
    for j in range(i+1, len(signature_matrix)):
        if signature_matrix[i] == signature_matrix[j]:
            similar_docs.append([i,j])

**Soru 2)**  Önceki soruda hazırladığınız LSH indeksini kullanarak verilen bir dosyanın muhtemel benzerleri olan dosyaları ve o dokümanlar ile olan jaccard benzerliği sıralı bir şekilde ekrana yazdıran bir method yazınız.  En çok benzeyen dokümanlar ilk yazılmalı. Çıktının formatı aşağıdaki örnekteki gibi olmalı.  Örnekte sadece 2 dosya döndüğü varsayılmıştır.

a.txt 0.7
b.txt 0.65
Toplam 2 dosya bulunmuştur.


In [15]:
# Cevabınız buraya yazınız. Eğer ihtiyaç duyarsanız metoda başka parametreler ekleyebilirsiniz
def query(input_file):  ... 

**Soru 3)** Bu soruda ödev ile verilen  metin dosyalarını kullanarak LSH indeksi kurup farklı k, b, ve r değerlerinin sonuç üzerindeki etkisine bakacaksınız.
Spesifik olarak yapacağınız şunlar.
- Ödevde verilen docs klasöründeki dokümanları b=5, r=4 iken k=2,3,4 değerlerini kullanarak 3 farklı LSH indeksi kurunuz. Ardından ödevde verilen sorgu.txt dosyasını bu 3 ayrı indekste aratıp toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz. 
- k=3, r=4 iken b değerini 3,4,5 olarak değiştiriniz ve birinci şıktaki gibi toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz. 
- k=3, r=4 iken r değerini 3,4,5 olarak değiştiriniz ve birinci şıktaki gibi toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz.

Not:  Bu soruda ikinci sorudaki metotta ufak bir değişikliğe gidip ekrana bir şey bastırmadan sadece  kaç doküman bulunduğunu dönen bir metot olarak tanımlayın. Bu yeni metodu aşağıda 3. soru için tanımlanan yerde yapın.

In [16]:
# 3. sorunun cevabı için ayrılan yer

**Soru 4**	Bu soruda MeanShift algoritmasını implement edeceksiniz. Yazacağınız metot verilen klasördeki her dosyayı meanShift algoritmasına ve radius uzaklık parametresine göre kümeler ve çıktı olarak kümeleri print eder. Algoritma en başta klasör içindeki her dosya için bir ağırlık merkezi başlatır. Çıktısı şu formatta olmalıdır.

Cluster-0: a,b,c

Cluster-1: d,e,f 

...........

Cluster-n: h
 
Yukarıdaki örnekte a,b,c, vs. verilen klasördeki dosyaların isimleridir. Dokümanların vektörel gösterimi şöyle olmalıdır: Tüm dokümanlarda geçen en popüler 1000 kelimeyi bulun. Sonra her kelimenin varlığı/yokluğunu birer öznitelik olarak gösterin. Örneğin bir doküman “a b c d f” ise ve b, d ve s en popüler ‘kelimeler’ ise, bu dokümanın gösterimi [1,1,0] olmalıdır. Uzaklık için kosinüs uzaklığını kullanın.



In [17]:
def meanShift (String folder_name, float radius):

SyntaxError: invalid syntax (1091480408.py, line 1)

**Soru 5** 4. soruda tanımladığınız metodu kullanarak ödev ile beraber verilen docs klasöründeki dosyaları cluster ediniz. Bunun için radius değerinini 0.1’den başlayıp 0.05 arttırarak 0.95’e kadar tüm değerlerde deneyin ve elde ettiğiniz cluster sayısını gösteriniz. Bunun için x ekseninde Radius değeri, y ekseninde cluster sayısı olan bir figür oluşturunuz.